In [1]:
import os
import torch
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes, save_image
from PIL import Image  # For optional thumbnail or display if needed

# Load pre-trained object detection model
model = fasterrcnn_resnet50_fpn(weights="DEFAULT")  # Downloads weights on first run
model.eval()  # Set to evaluation mode

# Transformation for input (model expects normalized tensors)
transform = transforms.Compose([
    transforms.ToTensor(),
])

def analyze_image(image_path, confidence_threshold=0.5):
    # Load image
    image = read_image(image_path)
    image_tensor = transform(Image.open(image_path).convert("RGB"))

    # Run detection
    with torch.no_grad():
        predictions = model([image_tensor])[0]

    # Filter boxes by confidence
    high_conf_idx = predictions['scores'] > confidence_threshold
    boxes = predictions['boxes'][high_conf_idx]
    labels = predictions['labels'][high_conf_idx]

    if len(boxes) == 0:
        print(f"No subjects detected in {image_path}")
        return None

    # Draw boxes on image
    image_with_boxes = draw_bounding_boxes(image, boxes, width=5, colors="red")

    # Save the output
    output_path = image_path.replace('.jpg', '_boxed.jpg')  # Adjust extension if needed
    save_image(image_with_boxes.float() / 255, output_path)
    print(f"Saved boxed image to {output_path}")

    # Compute positions for patterns (normalized centers)
    height, width = image.shape[1], image.shape[2]  # H, W from tensor
    centers = []
    for box in boxes:
        xmin, ymin, xmax, ymax = box
        center_x = ((xmin + xmax) / 2) / width
        center_y = ((ymin + ymax) / 2) / height
        centers.append((center_x.item(), center_y.item()))

    return centers

# Main function to process a folder
def analyze_folder(folder_path):
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    all_centers = []

    for file in image_files:
        path = os.path.join(folder_path, file)
        centers = analyze_image(path)
        if centers:
            all_centers.extend(centers)
            print(f"Centers for {file}: {centers}")

    if all_centers:
        # Basic pattern analysis (e.g., average position)
        avg_x = sum(c[0] for c in all_centers) / len(all_centers)
        avg_y = sum(c[1] for c in all_centers) / len(all_centers)
        print(f"\nOverall patterns across {len(image_files)} images:")
        print(f"- Average subject center: x={avg_x:.2f} (0=left, 1=right), y={avg_y:.2f} (0=top, 1=bottom)")
        if avg_x < 0.4:
            print("- Tendency to place subjects on the left.")
        elif avg_x > 0.6:
            print("- Tendency to place subjects on the right.")
        else:
            print("- Subjects often centered horizontally.")
        if avg_y < 0.4:
            print("- Subjects often in the upper part of the frame.")
        elif avg_y > 0.6:
            print("- Subjects often in the lower part of the frame.")
        else:
            print("- Subjects often centered vertically.")

folder_path = "/content/"  # Change to your folder
analyze_folder(folder_path)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


100%|██████████| 160M/160M [00:01<00:00, 162MB/s]


Saved boxed image to /content/PXL_20230909_230220242_boxed.jpg
Centers for PXL_20230909_230220242.jpg: [(0.17665913701057434, 0.6796562671661377), (0.49221178889274597, 0.6882927417755127), (0.7483956217765808, 0.6953920722007751), (0.8390349745750427, 0.6998414993286133), (0.39303773641586304, 0.686179518699646), (0.6381164789199829, 0.6930094957351685), (0.3017145097255707, 0.6844546794891357)]
No subjects detected in /content/PXL_20250308_051545003.jpg
Saved boxed image to /content/PXL_20240123_074008546_boxed.jpg
Centers for PXL_20240123_074008546.jpg: [(0.865150511264801, 0.5346399545669556), (0.7489351630210876, 0.5015894174575806), (0.7444515824317932, 0.4990505576133728), (0.7415317296981812, 0.49647387862205505), (0.7453347444534302, 0.5042505264282227)]

Overall patterns across 3 images:
- Average subject center: x=0.62 (0=left, 1=right), y=0.61 (0=top, 1=bottom)
- Tendency to place subjects on the right.
- Subjects often in the lower part of the frame.


In [ ]:
import os
import numpy as np
import scipy.ndimage as ndimage
from torchvision.utils import draw_bounding_boxes, save_image
from torchvision.io import read_image
from torchvision.transforms import Resize
import torch

def get_centroids(small_data, k=8, max_iters=10):
    n = small_data.shape[0]
    centroids = small_data[torch.randperm(n)[:k]]
    for _ in range(max_iters):
        dist = torch.cdist(small_data, centroids)
        assignments = torch.argmin(dist, dim=1)
        for j in range(k):
            mask = (assignments == j)
            if mask.sum() > 0:
                centroids[j] = small_data[mask].mean(dim=0)
    return centroids

def analyze_image(image_path, confidence_threshold=0.001):
    image = read_image(image_path)
    img_tensor = image.float() / 255.0
    img_np = img_tensor.permute(1, 2, 0).numpy()
    height, width = img_np.shape[:2]
    min_area = confidence_threshold * height * width  # Threshold as fraction of image size
    img_flat = img_np.reshape(-1, 3)
    # Downsample for faster clustering
    small_size = (height // 4, width // 4)
    resize = Resize(small_size)
    small_img = resize(img_tensor)
    small_img_np = small_img.permute(1, 2, 0).numpy()
    small_data = torch.from_numpy(small_img_np.reshape(-1, 3)).float()
    centroids = get_centroids(small_data)
    data = torch.from_numpy(img_flat).float()
    dist = torch.cdist(data, centroids)
    assignments = torch.argmin(dist, dim=1)
    labels = assignments.reshape(height, width).numpy()
    all_boxes = []
    all_box_colors = []
    all_centers = []
    for lab in np.unique(labels):
        mask = (labels == lab)
        component_labels, num_components = ndimage.label(mask)
        for comp in range(1, num_components + 1):
            positions = np.argwhere(component_labels == comp)
            if len(positions) < min_area:
                continue
            min_y, min_x = positions.min(axis=0)
            max_y, max_x = positions.max(axis=0)
            box = torch.tensor([min_x, min_y, max_x, max_y])
            all_boxes.append(box)
            color = tuple(int(c.item() * 255) for c in centroids[lab])
            all_box_colors.append(color)
            # Center for patterns
            center_x = ((min_x + max_x) / 2) / width
            center_y = ((min_y + max_y) / 2) / height
            all_centers.append((center_x, center_y))
    if not all_boxes:
        print(f"No color shapes detected in {image_path}")
        return None
    boxes = torch.stack(all_boxes)
    image_with_boxes = draw_bounding_boxes(image, boxes, colors=all_box_colors, width=3)
    output_path = image_path.replace('.jpg', '_color_boxes.jpg')  # Adjust extension if needed
    save_image(image_with_boxes.float() / 255, output_path)
    print(f"Saved boxed image to {output_path}")
    return all_centers

# Main function to process a folder
def analyze_folder(folder_path):
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    all_centers = []

    for file in image_files:
        path = os.path.join(folder_path, file)
        centers = analyze_image(path)
        if centers:
            all_centers.extend(centers)
            print(f"Centers for {file}: {centers}")

    if all_centers:
        # Basic pattern analysis
        avg_x = sum(c[0] for c in all_centers) / len(all_centers)
        avg_y = sum(c[1] for c in all_centers) / len(all_centers)
        print(f"\nOverall patterns across {len(image_files)} images:")
        print(f"- Average color shape center: x={avg_x:.2f} (0=left, 1=right), y={avg_y:.2f} (0=top, 1=bottom)")
        if avg_x < 0.4:
            print("- Tendency to place color shapes on the left.")
        elif avg_x > 0.6:
            print("- Tendency to place color shapes on the right.")
        else:
            print("- Color shapes often centered horizontally.")
        if avg_y < 0.4:
            print("- Color shapes often in the upper part of the frame.")
        elif avg_y > 0.6:
            print("- Color shapes often in the lower part of the frame.")
        else:
            print("- Color shapes often centered vertically.")


folder_path = "/content/"  # Change to your folder
analyze_folder(folder_path)